In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

  # Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00


In [2]:
from pathlib import Path
# wandb library
import wandb
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn import metrics
import torch
torch.cuda.empty_cache()
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed
from transformers import T5ForSequenceClassification, AutoTokenizer


import datasets
from datasets import Dataset, DatasetDict

import evaluate



!git lfs install

Git LFS initialized.


In [3]:
base_folder = Path(basepath)
data_folder = base_folder/'Homework/Homework7'
model_folder = base_folder/'models/nlp_spring_2024/ed/nn'
custom_functions = base_folder/'custom-functions'

In [4]:
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')
import CustomPreprocessorSpacy as cp

In [5]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [6]:
train_data = pd.read_csv(data_folder/ 'train.csv')
test= pd.read_csv(data_folder/ 'test.csv')

In [7]:
train_data.drop(columns=["ID"],inplace=True)
y_test = test.drop(['ID','Tweet'],axis=1)
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(int, x)))
test_data = pd.concat([test['Tweet'],y_test],axis=1)

train_data.head()


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [8]:
test_data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,0,0,0,0,0,0,0,0,0,0,0
1,"Academy of Sciences, eschews the normally sobe...",0,0,0,0,0,0,0,0,0,0,0
2,I blew that opportunity -__- #mad,0,0,0,0,0,0,0,0,0,0,0
3,This time in 2 weeks I will be 30... 😥,0,0,0,0,0,0,0,0,0,0,0
4,#Deppression is real. Partners w/ #depressed p...,0,0,0,0,0,0,0,0,0,0,0


In [9]:
labels = [x for x in train_data.columns if x not in ['Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
NUM_OF_LABELS = len(labels)

In [10]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_testEval = train_dataset.train_test_split(train_size=0.80)
val_eval = train_testEval['test']

ds = DatasetDict({
    'train' : train_testEval['train'],
    'val' : val_eval,
    'test':test_dataset

})

print("Training Dataset Shape:", ds['train'].shape)
print("Evaluation Dataset Shape:", ds['val'].shape)
print("Testing Dataset Shape",ds['test'].shape)
ds['train'][25]

Training Dataset Shape: (6179, 12)
Evaluation Dataset Shape: (1545, 12)
Testing Dataset Shape (3259, 12)


{'Tweet': 'Food that gets delivered 😍🙌🏻 #cheering #happy',
 'anger': 0,
 'anticipation': 0,
 'disgust': 0,
 'fear': 0,
 'joy': 1,
 'love': 0,
 'optimism': 1,
 'pessimism': 0,
 'sadness': 0,
 'surprise': 1,
 'trust': 0}

In [11]:
set_seed = 42
NUM_OF_EPOCHS = 10

BATCH_SIZE = 32
LEARNING_RATE = 1e-4

MAX_LENGTH = 128
MODEL_NAME = "t5-base"

METRIC_NAME = "f1_macro"
DEVICE = torch.device("cpu")
print(f"This project will utilize the {DEVICE} to train and evaluate.")
print(f"This project will have a maximum length for the input text of {MAX_LENGTH} tokens.")

This project will utilize the cpu to train and evaluate.
This project will have a maximum length for the input text of 128 tokens.


In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def data_preprocessor(examples):
	# take a batch of text
	text = examples["Tweet"]
	# encode them
	encoded_data = tokenizer(text, padding="max_length", truncation=True, max_length=MAX_LENGTH)
	# add labels
	labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
	# create numpy array of shape (batch_size, num_labels)
	labels_matrix = np.zeros((len(text), len(labels)))
	# fill numpy array
	for idx, label in enumerate(labels):
		labels_matrix[:, idx] = labels_batch[label]

	encoded_data["labels"] = labels_matrix.tolist()

	return encoded_data

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:
encoded_ds = ds.map(data_preprocessor, batched=True, load_from_cache_file=False)
encoded_ds.set_format("torch")

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [14]:
print(f"The shape of the training dataset is: {encoded_ds['train'].shape}")

print(f"The shape of the evaluation dataset is: {encoded_ds['val'].shape}")
print(f"The shape of the test dataset is: {encoded_ds['test'].shape}")

The shape of the training dataset is: (6179, 15)
The shape of the evaluation dataset is: (1545, 15)
The shape of the test dataset is: (3259, 15)


In [15]:
model = (T5ForSequenceClassification.from_pretrained(MODEL_NAME,
                                                             num_labels=NUM_OF_LABELS,
                                                             problem_type="multi_label_classification",
                                                             id2label=id2label,
                                                             label2id=label2id)
         ).to(DEVICE)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def multi_label_metrics(predictions,
                        labels,
                        threshold=0.5):
    '''
    This function calculates & returns metrics
    for a multilabel classification analysis.
    '''

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true,y_pred=y_pred,average ='macro')

    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return metrics as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro':f1_macro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds = preds[0] if isinstance(preds,
                    tuple) else preds
    results = multi_label_metrics(
                    predictions=preds,
                    labels=labels)
    return results

In [17]:
args = TrainingArguments(
    output_dir=model_folder,
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=100,
    save_total_limit=1,# Save a model checkpoint every 10 training steps
    learning_rate=LEARNING_RATE,
    lr_scheduler_type='linear',
    num_train_epochs=NUM_OF_EPOCHS,
    optim='adamw_torch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    warmup_steps = 0,
    greater_is_better=True,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_first_step=True,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Tweet_Emotion_FLANT5',
    fp16=True
)

In [18]:
trainer = Trainer(
    model,
    args,

    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [20]:
trainer.train()

wandb: Currently logged in as: rangareddy-bhargav. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
50,0.686200,0.474177,0.003249,0.001904,0.025243
100,0.686200,0.402735,0.449932,0.221011,0.148220
150,0.686200,0.363697,0.559313,0.301985,0.174757
200,0.686200,0.342519,0.622324,0.373472,0.208414
250,0.686200,0.324235,0.640100,0.443611,0.233657
300,0.686200,0.313199,0.663503,0.464431,0.260194
350,0.686200,0.304569,0.661712,0.470971,0.258900
400,0.686200,0.303732,0.675516,0.482677,0.248544
450,0.686200,0.302731,0.675180,0.490028,0.258900
500,0.348700,0.300320,0.682687,0.503053,0.270550


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=1940, training_loss=0.25013460129806675, metrics={'train_runtime': 2020.4157, 'train_samples_per_second': 30.583, 'train_steps_per_second': 0.96, 'total_flos': 9435304811896320.0, 'train_loss': 0.25013460129806675, 'epoch': 10.0})

In [21]:
trainer.evaluate()

{'eval_loss': 0.3111904263496399,
 'eval_f1_micro': 0.702656383890317,
 'eval_f1_macro': 0.5745047009285973,
 'eval_accuracy': 0.27119741100323624,
 'eval_runtime': 10.6147,
 'eval_samples_per_second': 145.553,
 'eval_steps_per_second': 4.616,
 'epoch': 10.0}

In [22]:
valid_output = trainer.predict(encoded_ds["val"])

In [23]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [24]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 1500.


In [25]:
model_folder

PosixPath('/content/drive/MyDrive/NLP/models/nlp_spring_2024/ed/nn')

In [26]:
model_checkpoint = str(model_folder) + '/checkpoint-' + best_model_checkpoint_step

In [27]:
valid_results = trainer.evaluate(encoded_ds["val"])

In [28]:
valid_results

{'eval_loss': 0.3111904263496399,
 'eval_f1_micro': 0.702656383890317,
 'eval_f1_macro': 0.5745047009285973,
 'eval_accuracy': 0.27119741100323624,
 'eval_runtime': 10.3588,
 'eval_samples_per_second': 149.149,
 'eval_steps_per_second': 4.73,
 'epoch': 10.0}

In [29]:
model = T5ForSequenceClassification.from_pretrained(model_checkpoint)
model.to('cuda')
model.eval()

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feat

In [30]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_ds['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [32]:
encoded_ds['test']['labels']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [33]:
test_results = trainer.evaluate()

In [34]:
test_results = trainer.predict(encoded_ds['test'])

# The predictions object contains various fields including predictions
logits = test_results.predictions[0]

# Apply sigmoid to get the probabilities
probs = torch.sigmoid(torch.Tensor(logits))

# Apply a threshold to get the binary predictions
threshold = 0.5
y_pred = (probs >= threshold).int().numpy()

In [35]:
test_results.predictions

(array([[ 1.0439453 , -3.0683594 ,  0.43774414, ..., -1.8896484 ,
         -5.5664062 , -5.1015625 ],
        [ 1.1445312 , -3.7539062 ,  1.53125   , ...,  0.38305664,
         -5.1445312 , -6.6328125 ],
        [ 5.2734375 , -4.9375    ,  3.6992188 , ...,  0.3803711 ,
         -4.4921875 , -7.4414062 ],
        ...,
        [ 6.9023438 , -4.234375  ,  4.2539062 , ..., -2.0917969 ,
         -5.2304688 , -6.1132812 ],
        [-4.0976562 , -2.4511719 , -4.2265625 , ..., -4.859375  ,
         -1.5996094 , -1.3115234 ],
        [-1.3867188 , -2.5371094 , -1.1494141 , ..., -3.6328125 ,
         -3.4960938 , -4.0429688 ]], dtype=float32),
 array([[[ 1.03530865e-02,  3.93101610e-02,  2.33311012e-01, ...,
           5.25893629e-01, -1.01575479e-01,  2.71867514e-02],
         [-5.22031963e-01,  1.27154544e-01,  5.30952632e-01, ...,
           3.16550165e-01,  4.15139310e-02, -6.83843717e-02],
         [-7.54108667e-01,  1.22005403e-01,  4.68024433e-01, ...,
           4.59500909e-01,  4.621659

In [36]:
y_pred_df = pd.DataFrame(y_pred,columns=labels)
output_submission = pd.concat([test['ID'],y_pred_df],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,1,0,1,0,0,0,0,0,1,0,0
3,2018-03001,0,1,0,0,1,0,1,0,0,0,0
4,2018-01988,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,0,1,0,1,1,0,0,0,0,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [37]:
output_submission.to_csv('/content/drive/MyDrive/NLP/output_submission.csv', index=False)

In [38]:
wandb.finish()

eval/accuracy,▁▄▅▆▇██▇████████████████████████████████
eval/f1_macro,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████████
eval/f1_micro,▁▅▇▇▇███████████████████████████████████
eval/loss,█▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂
eval/runtime,█▄▄▂▆▁▅▂▄▁▄▁▅▂▅▁▄▂▅▁▆▂▅▂▅▂▅▂▇▂▇▃▅▂▅▃▅▂▅▃
eval/samples_per_second,▁▅▅▇▃█▄▇▄█▄█▄▇▄█▄▇▄█▃▇▃▇▄▇▄▇▂▇▂▆▄▇▄▆▄▇▃▅
eval/steps_per_second,▁▅▅▇▃█▄▇▄█▄█▄▇▄█▄▇▄█▃▇▃▇▄▇▄▇▂▇▂▆▄▇▄▆▄▇▃▅
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,█▂▁▃
train/learning_rate,█▆▃▁
